In [88]:
#import libraries
import pandas as pd
import numpy as np
import psycopg2
import pyspark
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import month, dayofmonth, dayofweek, hour, date_format
from pyspark.sql.functions import avg, unix_timestamp, StringType, udf, coalesce, lit
from pyspark.sql import functions as F

# Extract data from CSV files

In [56]:
spark = SparkSession.builder.appName("Proj").getOrCreate()

## contacts.csv

In [57]:
df_con = spark.read.option('header', 'true').csv("data/contacts.csv", inferSchema=True)
df_con.show(5)

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+----------------+-----------------+--------+--------------+------------------------------------+---------------------+----------------------+
|       id_guest_anon|        id_host_anon|     id_listing_anon|ts_interaction_first|  ts_reply_at_first|ts_accepted_at_first|      ts_booking_at|ds_checkin_first|ds_checkout_first|m_guests|m_interactions|m_first_message_length_in_characters|contact_channel_first|guest_user_stage_first|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+----------------+-----------------+--------+--------------+------------------------------------+---------------------+----------------------+
|da8656a1-51af-4f3...|5426897d-960d-401...|a408a8b2-0d44-451...| 2016-04-21 02:55:53|2016-04-21 03:15:00| 2016-04-21 03:15:00|2016-04-21

In [58]:
df_con.printSchema()

root
 |-- id_guest_anon: string (nullable = true)
 |-- id_host_anon: string (nullable = true)
 |-- id_listing_anon: string (nullable = true)
 |-- ts_interaction_first: timestamp (nullable = true)
 |-- ts_reply_at_first: timestamp (nullable = true)
 |-- ts_accepted_at_first: timestamp (nullable = true)
 |-- ts_booking_at: timestamp (nullable = true)
 |-- ds_checkin_first: date (nullable = true)
 |-- ds_checkout_first: date (nullable = true)
 |-- m_guests: double (nullable = true)
 |-- m_interactions: integer (nullable = true)
 |-- m_first_message_length_in_characters: double (nullable = true)
 |-- contact_channel_first: string (nullable = true)
 |-- guest_user_stage_first: string (nullable = true)



In [59]:
count_dup_rows = df_con.count() - df_con.distinct().count()
print(f"Number of duplicate rows: {count_dup_rows}")


Number of duplicate rows: 0


In [60]:
Dict_Null = {col:df_con.filter(df_con[col].isNull()).count() for col in df_con.columns}
print(f"Number of Null values in each column: ")
for key, value in Dict_Null.items():
    print(key, ": ", value)

Number of Null values in each column: 
id_guest_anon :  0
id_host_anon :  0
id_listing_anon :  0
ts_interaction_first :  0
ts_reply_at_first :  2032
ts_accepted_at_first :  11472
ts_booking_at :  16300
ds_checkin_first :  0
ds_checkout_first :  0
m_guests :  1
m_interactions :  0
m_first_message_length_in_characters :  0
contact_channel_first :  0
guest_user_stage_first :  0


## listings.csv

In [61]:
df_lis = spark.read.option('header', 'true').csv("data/listings.csv", inferSchema=True)
df_lis.show(5)

+--------------------+---------------+--------------------+-------------+
|     id_listing_anon|      room_type|listing_neighborhood|total_reviews|
+--------------------+---------------+--------------------+-------------+
|71582793-e5f8-46d...|   Private room|           -unknown-|          0.0|
|a1a3f728-e21f-443...|Entire home/apt|          Copacabana|          0.0|
|353a68be-ecf9-4b7...|Entire home/apt|     Barra da Tijuca|          3.0|
|b9ae1908-0486-40a...|Entire home/apt|                Lapa|          4.0|
|fa0290ef-7881-448...|Entire home/apt|           -unknown-|          0.0|
+--------------------+---------------+--------------------+-------------+
only showing top 5 rows



In [62]:
df_lis.printSchema()

root
 |-- id_listing_anon: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- listing_neighborhood: string (nullable = true)
 |-- total_reviews: double (nullable = true)



In [63]:
count_dup_rows = df_lis.count() - df_lis.distinct().count()
print(f"Number of duplicate rows: {count_dup_rows}")

Number of duplicate rows: 0


In [64]:
Dict_Null = {col:df_lis.filter(df_lis[col].isNull()).count() for col in df_lis.columns}
print(f"Number of Null values in each column: ")
for key, value in Dict_Null.items():
    print(key, ": ", value)

Number of Null values in each column: 
id_listing_anon :  0
room_type :  0
listing_neighborhood :  0
total_reviews :  0


## users.csv

In [65]:
df_user = spark.read.option('header', 'true').csv("data/users.csv", inferSchema=True)
df_user.show(5)

+--------------------+-------+---------------------+
|        id_user_anon|country|words_in_user_profile|
+--------------------+-------+---------------------+
|1d16a001-31a2-494...|     FR|                    0|
|42607e0a-86c0-472...|     AR|                    0|
|25f85eb5-a700-44e...|     BR|                    0|
|55abeba0-18ef-4c5...|     BR|                    1|
|5d62d35a-7d6d-45d...|     BR|                   98|
+--------------------+-------+---------------------+
only showing top 5 rows



In [66]:
df_user.printSchema()

root
 |-- id_user_anon: string (nullable = true)
 |-- country: string (nullable = true)
 |-- words_in_user_profile: integer (nullable = true)



In [67]:
count_dup_rows = df_user.count() - df_user.distinct().count()
print(f"Number of duplicate rows: {count_dup_rows}")

Number of duplicate rows: 68


In [68]:
df_user = df_user.dropDuplicates()

In [69]:
count_dup_rows = df_user.count() - df_user.distinct().count()
print(f"Number of duplicate rows: {count_dup_rows}")

Number of duplicate rows: 0


In [70]:
Dict_Null = {col:df_user.filter(df_user[col].isNull()).count() for col in df_user.columns}
print(f"Number of Null values in each column: ")
for key, value in Dict_Null.items():
    print(key, ": ", value)

Number of Null values in each column: 
id_user_anon :  0
country :  0
words_in_user_profile :  0


## Explore Listings

In [71]:
df_lis.orderBy("total_reviews", ascending=False).select("listing_neighborhood", "room_type", "total_reviews").show()

+--------------------+---------------+-------------+
|listing_neighborhood|      room_type|total_reviews|
+--------------------+---------------+-------------+
|        Santa Teresa|   Private room|        268.0|
|             Ipanema|Entire home/apt|        209.0|
|          Copacabana|Entire home/apt|        185.0|
|        Santa Teresa|Entire home/apt|        182.0|
|          Copacabana|Entire home/apt|        182.0|
|          Copacabana|Entire home/apt|        174.0|
|        Santa Teresa|   Private room|        165.0|
|                Lapa|Entire home/apt|        163.0|
|          Copacabana|Entire home/apt|        162.0|
|            Botafogo|   Private room|        162.0|
|          Copacabana|Entire home/apt|        159.0|
|          Copacabana|Entire home/apt|        150.0|
|               Lagoa|Entire home/apt|        149.0|
|             Ipanema|   Private room|        148.0|
|          Copacabana|Entire home/apt|        147.0|
|        Santa Teresa|Entire home/apt|        

In [72]:
df_lis.groupBy("room_type").count().show()

+---------------+-----+
|      room_type|count|
+---------------+-----+
|    Shared room|  372|
|Entire home/apt| 9647|
|   Private room| 3019|
+---------------+-----+



In [73]:
df_lis.groupBy("listing_neighborhood").count().sort("count", ascending=False).show()

+--------------------+-----+
|listing_neighborhood|count|
+--------------------+-----+
|           -unknown-| 6221|
|          Copacabana| 2531|
|             Ipanema| 1041|
|     Barra da Tijuca|  593|
|              Leblon|  458|
|            Botafogo|  345|
|        Santa Teresa|  243|
|            Flamengo|  171|
|                Lapa|  171|
|                Leme|  168|
|Recreio dos Bande...|  164|
|         Laranjeiras|  120|
|              Tijuca|   82|
|               Lagoa|   68|
|              Glória|   68|
|              Catete|   59|
|               Gávea|   54|
|     Jardim Botânico|   52|
|              Centro|   51|
|             Humaitá|   43|
+--------------------+-----+
only showing top 20 rows



## Explore users

In [74]:
df_user.groupBy("country").count().sort("count", ascending=False).show()

+-------+-----+
|country|count|
+-------+-----+
|     BR|19568|
|     US| 2878|
|     AR| 1770|
|     FR| 1164|
|     GB|  975|
|     DE|  610|
|     CL|  410|
|     CA|  387|
|     AU|  367|
|     NL|  271|
|     ES|  243|
|     CO|  241|
|     IT|  228|
|     CH|  227|
|     UY|  206|
|     MX|  173|
|     PE|  119|
|     PT|  110|
|     BE|  100|
|     DK|   77|
+-------+-----+
only showing top 20 rows



## Join all tables

In [75]:
df_joined = df_con.join(df_user, df_user.id_user_anon == df_con.id_guest_anon)
df = df_joined.join(df_lis, df_lis.id_listing_anon == df_joined.id_listing_anon) 
df.show(1)

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+----------------+-----------------+--------+--------------+------------------------------------+---------------------+----------------------+--------------------+-------+---------------------+--------------------+---------------+--------------------+-------------+
|       id_guest_anon|        id_host_anon|     id_listing_anon|ts_interaction_first|  ts_reply_at_first|ts_accepted_at_first|      ts_booking_at|ds_checkin_first|ds_checkout_first|m_guests|m_interactions|m_first_message_length_in_characters|contact_channel_first|guest_user_stage_first|        id_user_anon|country|words_in_user_profile|     id_listing_anon|      room_type|listing_neighborhood|total_reviews|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+----------------+---------------

## Are there seasonal trends in booking times, check-ins, or interactions?

In [76]:
df_trends = df.select(
    col("ts_booking_at").cast("timestamp").alias("booking_time"),
    col("ds_checkin_first").cast("date").alias("checkin_date"),
    col("ts_interaction_first").cast("timestamp").alias("interaction_time")
)

df_trends = df_trends.withColumn("booking_month", month("booking_time")) \
                     .withColumn("booking_weekday", dayofweek("booking_time")) \
                     .withColumn("checkin_month", month("checkin_date")) \
                     .withColumn("interaction_month", month("interaction_time")) \
                     .withColumn("interaction_hour", hour("interaction_time"))


# Booking trends by month
booking_trends = df_trends.groupBy("booking_month") \
    .agg(count("*").alias("total_bookings")) \
    .orderBy("total_bookings", ascending=False)
print("Booking Trends by Month:")
booking_trends.show()

# Check-in trends by month
checkin_trends = df_trends.groupBy("checkin_month") \
    .agg(count("*").alias("total_checkins")) \
    .orderBy("total_checkins", ascending=False)
print("Check-In Trends by Month:")
checkin_trends.show()

# Interaction trends by month and hour
interaction_trends = df_trends.groupBy("interaction_month", "interaction_hour") \
    .agg(count("*").alias("total_interactions")) \
    .orderBy("total_interactions", ascending=False)
print("Interaction Trends by Month and Hour:")
interaction_trends.show(24)

Booking Trends by Month:
+-------------+--------------+
|booking_month|total_bookings|
+-------------+--------------+
|         NULL|         16300|
|            6|          2167|
|            1|          2103|
|            5|          2047|
|            2|          1796|
|            3|          1764|
|            4|          1675|
|            7|            28|
|            8|             6|
|            9|             1|
+-------------+--------------+

Check-In Trends by Month:
+-------------+--------------+
|checkin_month|total_checkins|
+-------------+--------------+
|            8|          9455|
|            2|          5185|
|            3|          2497|
|            4|          2385|
|            5|          2114|
|            6|          1811|
|            7|          1807|
|            1|          1365|
|           12|           638|
|            9|           350|
|           10|           176|
|           11|           104|
+-------------+--------------+

Interaction Trend

## How does the timing of interactions affect the success rate of bookings?

In [77]:
df_timing = df.withColumn("interaction_time", unix_timestamp("ts_interaction_first")) \
              .withColumn("booking_time", unix_timestamp("ts_booking_at")) \
              .withColumn("is_successful", when(col("ts_booking_at").isNotNull(), 1).otherwise(0))

# Calculate the time difference (in hours) between interaction and booking
df_timing = df_timing.withColumn(
    "time_to_booking_hours",
    ((col("booking_time") - col("interaction_time")) / 3600).cast("double")
)

time_bins = [
    (-1, 0), (0, 1), (1, 6), (6, 12), (12, 24), (24, 72), (72, 168), (168, float("inf"))
]
time_labels = ["<0", "0-1h", "1-6h", "6-12h", "12-24h", "1-3d", "3-7d", ">7d"]

df_timing_filtered = df_timing.filter((col("time_to_booking_hours") >= 0) & 
                                      (col("time_to_booking_hours") <= 168))

df_timing = df_timing.withColumn(
    "time_to_booking_bucket",
    when(col("time_to_booking_hours") < 0, "<0")
    .when((col("time_to_booking_hours") >= 0) & (col("time_to_booking_hours") < 1), "0-1h")
    .when((col("time_to_booking_hours") >= 1) & (col("time_to_booking_hours") < 6), "1-6h")
    .when((col("time_to_booking_hours") >= 6) & (col("time_to_booking_hours") < 12), "6-12h")
    .when((col("time_to_booking_hours") >= 12) & (col("time_to_booking_hours") < 24), "12-24h")
    .when((col("time_to_booking_hours") >= 24) & (col("time_to_booking_hours") < 72), "1-3d")
    .when((col("time_to_booking_hours") >= 72) & (col("time_to_booking_hours") < 168), "3-7d")
    .otherwise(">7d")
)

# Aggregate success rates per bucket
success_rate_df = df_timing.groupBy("time_to_booking_bucket") \
    .agg(
        count("*").alias("total_interactions"),
        avg("is_successful").alias("booking_success_rate")
    )\
    .orderBy("total_interactions")

success_rate_df.show()


+----------------------+------------------+--------------------+
|time_to_booking_bucket|total_interactions|booking_success_rate|
+----------------------+------------------+--------------------+
|                    <0|               121|                 1.0|
|                  3-7d|               183|                 1.0|
|                  1-3d|               588|                 1.0|
|                 6-12h|               599|                 1.0|
|                12-24h|               719|                 1.0|
|                  1-6h|              1300|                 1.0|
|                  0-1h|              7943|                 1.0|
|                   >7d|             16434|0.008153827430935864|
+----------------------+------------------+--------------------+



## Which countries have the quickest average reply times or highest acceptance rates?



In [78]:
country_metrics = df.withColumn(
    "reply_time_hours",
    (col("ts_reply_at_first").cast("long") - col("ts_interaction_first").cast("long")) / 3600
)

country_metrics = country_metrics.groupBy("country").agg(
    avg("reply_time_hours").alias("average_reply_time_hours"),
    (count(when(col("ts_accepted_at_first").isNotNull(), 1)) / count("*")).alias("acceptance_rate")
)

country_metrics_sorted = country_metrics.orderBy("average_reply_time_hours", "acceptance_rate", ascending=False)
country_metrics_sorted.show()

+-------+------------------------+-------------------+
|country|average_reply_time_hours|    acceptance_rate|
+-------+------------------------+-------------------+
|     CD|       480.0097222222222|                0.0|
|     GE|      127.52055555555556|                0.0|
|     DO|        92.9183024691358| 0.5555555555555556|
|     RU|       69.46749305555558| 0.6136363636363636|
|     KW|       68.41583333333332|               0.25|
|     IN|       65.71635933806148|              0.625|
|     ZA|       65.57135620915035| 0.5384615384615384|
|     NA|       53.97138888888889|                0.0|
|     SE|      41.359402777777774| 0.7710843373493976|
|     LV|       39.32170634920635|0.42857142857142855|
|     DK|       37.81549019607843| 0.7608695652173914|
|     RE|      30.977500000000003|                1.0|
|     KE|       27.24347222222222| 0.6666666666666666|
|     MX|       26.94541666666666| 0.6428571428571429|
|     UA|      26.802329059829063|                0.6|
|     SM| 

## How does neighborhood popularity or guest interaction behavior vary across regions?

In [89]:
neighborhood_analysis = df.groupBy("listing_neighborhood").agg(
    F.countDistinct("id_guest_anon").alias("unique_guests"),
    F.avg("m_interactions").alias("avg_interactions_per_guest"),
    F.avg("m_first_message_length_in_characters").alias("avg_first_message_length"),
    F.avg("m_guests").alias("avg_guests_per_booking"),
    F.avg("total_reviews").alias("avg_reviews"),
    F.avg("ts_booking_at").alias("avg_booking_time")
).orderBy("unique_guests", ascending=False)
print("Neighborhood_analysis: Grouping by neighborhood and calculating aggregated metrics:")
neighborhood_analysis.show()


country_analysis = df.groupBy("country").agg(
    F.avg("m_interactions").alias("avg_interactions"),
    F.avg("m_guests").alias("avg_guests_per_booking"),
    F.avg("total_reviews").alias("avg_reviews"),
)

print("Show the country-level interaction analysis:")
country_analysis.show()


Neighborhood_analysis: Grouping by neighborhood and calculating aggregated metrics:
+--------------------+-------------+--------------------------+------------------------+----------------------+------------------+--------------------+
|listing_neighborhood|unique_guests|avg_interactions_per_guest|avg_first_message_length|avg_guests_per_booking|       avg_reviews|    avg_booking_time|
+--------------------+-------------+--------------------------+------------------------+----------------------+------------------+--------------------+
|           -unknown-|        10376|         8.040750760906324|       191.8834122421373|    2.8143231588737634| 8.233513696313832|1.4598606301302834E9|
|          Copacabana|         5980|         8.431132220009193|       197.1388080281906|    2.9149685920024515|22.292783821051017|1.4593859040534723E9|
|             Ipanema|         2821|         8.225784903139612|      188.68470273881096|    2.8537074148296595|27.597862391449567|1.4592580806494699E9|
|   